20240620 version

### 測定データの可視化 + Schottky fitting

1. ファイルが入っているフォルダーを指定
2. ファイルを読み込んでデータの選別
3. データの可視化
4. 対象となるデータを可視化して、解析領域（x軸の値を指定）を手動で決める
5. Schottky moduleを使ってFitting


In [ ]:
import datetime
import os
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore')
# warnings.resetwarnings()
# warnings.simplefilter('ignore', FutureWarning)

# このモジュールで処理が行われる
from ivlib import IV_reader as ivr
from ivlib import schottky as stk

%load_ext autoreload
%autoreload 2

In [ ]:
# 読み込みたいフォルダーを絶対パスで指定する
# その際, r" "としてrを書く。このrはRawの略 特にWindowsでパスを書くときは必要

# all_files=[r"C:\........\IGB97", 
#            r"C:\........\IGB98"]

#　List形で書いてください。  
all_files=[]

In [ ]:
# ファイルフォルダーを指定してデータを読み込みます。
igb = ivr.IVReader(all_files[0])

# 読み込んだファイルフォルダーの確認
igb.file_folder

In [ ]:
# データを読み込んでデータの選別を行います。
# All：すべてのデータ、Contact：オーミックなふるまいのデータ、Noise:ノイズだけのデータ、Select:Contact,Noise以外のデータ
igb.read_file()

In [ ]:
igb.one_fig_plots(all_or_select='all',save=False) # all --> 'all' select --> 'select'

# 1つ１つのグラフとして表示する場合は以下のメソッドを利用（後でコピーペーストで再利用しやすい）
# igb.each_plots(all_or_select='select')

In [ ]:
# 解析対象のデータを見る
# SelectデータのIndex=5のデータ
# (-5V -> 5V,5V -> -5V)の連続データが入っている
# dfa -> -5V-->0V-->5V,
# dfb-> 5V-->0V-->-5V

dfa,dfb = igb.focuse_data('select', index=5)

In [ ]:
# これ以降Schottky解析
# from ivlib import schottky as stk

# 電極サイズを指定
A_eff=7.85e-5

# グラフから解析範囲を切り出します。（x軸の値を指定する）
xf2,yf2 = stk.select_xarray(dfa['V1'].values,dfa['I1'].values,xmin=1.5,xmax=3.0)

# 必要があれば電極サイズで割ってJ [A/cm2]に変換
yf2 = yf2/A_eff

res = stk.analysis_all(V=xf2, J=yf2, A_eff=A_eff, AS=36, T=300, plot=True)